### Links To CDC for Raw Data

## Covid-19 Infection Data
### url-1: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

## Covid-19 Vaccine Distribution
### url-2: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [33]:
# import dependencies
import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, inspect, func
import psycopg2
import requests
from sodapy import Socrata

# import cdc key for API
from cdc_token import my_token

## USA Monthly Cases by State 

In [34]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9mfq-cb36", limit=50000)

# Convert to pandas DataFrame and display
covid_df = pd.DataFrame.from_records(results)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death
46795,2020-12-11T00:00:00.000,AZ,394804,380243.0,14561,6986.0,459,7245,91.0,12,2020-12-12T14:59:37.544,Agree,Agree,6689.0,556
46796,2021-08-13T00:00:00.000,FSM,7,7.0,0,0.0,0,0,0.0,0,2021-08-14T14:46:52.851,Agree,Agree,0.0,0
46797,2020-12-15T00:00:00.000,DC,25339,NaN,NaN,301.0,0,720,4.0,0,2020-12-16T14:28:57.146,NaN,NaN,NaN,NaN
46798,2021-07-06T00:00:00.000,OR,209494,209494.0,0,117.0,0,2782,1.0,0,2021-07-07T14:48:49.531,Agree,Agree,2782.0,0
46799,2021-11-22T00:00:00.000,AZ,1245127,1127692.0,117435,3249.0,403,21942,2.0,0,2021-11-23T14:18:53.686,Agree,Agree,19414.0,2528


In [35]:
# add date columns which only has year and month for use with groupby later on.. 
covid_df['date'] = pd.to_datetime(covid_df['submission_date']).dt.to_period('M').astype(str)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death,date
46795,2020-12-11T00:00:00.000,AZ,394804,380243.0,14561,6986.0,459,7245,91.0,12,2020-12-12T14:59:37.544,Agree,Agree,6689.0,556,2020-12
46796,2021-08-13T00:00:00.000,FSM,7,7.0,0,0.0,0,0,0.0,0,2021-08-14T14:46:52.851,Agree,Agree,0.0,0,2021-08
46797,2020-12-15T00:00:00.000,DC,25339,NaN,NaN,301.0,0,720,4.0,0,2020-12-16T14:28:57.146,NaN,NaN,NaN,NaN,2020-12
46798,2021-07-06T00:00:00.000,OR,209494,209494.0,0,117.0,0,2782,1.0,0,2021-07-07T14:48:49.531,Agree,Agree,2782.0,0,2021-07
46799,2021-11-22T00:00:00.000,AZ,1245127,1127692.0,117435,3249.0,403,21942,2.0,0,2021-11-23T14:18:53.686,Agree,Agree,19414.0,2528,2021-11


In [36]:
covid_df.dtypes

submission_date    object
state              object
tot_cases          object
conf_cases         object
prob_cases         object
new_case           object
pnew_case          object
tot_death          object
new_death          object
pnew_death         object
created_at         object
consent_cases      object
consent_deaths     object
conf_death         object
prob_death         object
date               object
dtype: object

In [37]:
# convert objects to float
covid_df["tot_cases"] =  covid_df.tot_cases.astype(float)
covid_df["tot_death"] = covid_df.tot_death.astype(float)

# select only the columns we gonna use and display df
covid_df = covid_df[["date", "state", "tot_cases", "tot_death"]]
covid_df.head()

,date,state,tot_cases,tot_death
0,2021-12,ND,163565.0,1907.0
1,2020-08,MD,100715.0,3765.0
2,2021-07,MD,464491.0,9822.0
3,2020-05,VT,855.0,52.0
4,2021-01,NC,706315.0,10066.0


In [6]:
# covid_test = covid_max.loc[(covid_max['state'] == 'TX')]
# covid_test.head(60)

In [12]:
# group by states to be used later
# state_df = covid_df.groupby('state').mean().reset_index()
# state_df.head()

In [38]:
# add total recovery columns to df
covid_df["Total_recovered"] = covid_df.tot_cases - covid_df.tot_death
covid_df.tail()

,date,state,tot_cases,tot_death,Total_recovered
46795,2020-12,AZ,394804.0,7245.0,387559.0
46796,2021-08,FSM,7.0,0.0,7.0
46797,2020-12,DC,25339.0,720.0,24619.0
46798,2021-07,OR,209494.0,2782.0,206712.0
46799,2021-11,AZ,1245127.0,21942.0,1223185.0


In [39]:
# groupby date and state and take the mean value to get monthly average..
covid_monthly_df = round(covid_df.groupby(["date", "state"]).max().reset_index(), 2)
covid_monthly_df

,date,state,tot_cases,tot_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,1.0,0.0,1.0
...,...,...,...,...,...
1615,2022-03,VT,105475.0,578.0,104897.0
1616,2022-03,WA,1437914.0,12183.0,1425731.0
1617,2022-03,WI,1577659.0,13713.0,1563946.0
1618,2022-03,WV,494875.0,6550.0,488325.0


In [40]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_monthly_df = covid_monthly_df.rename(columns={"date": "Date", "state": "State", "tot_cases": "Total_cases", "tot_death": "Total_death"})
covid_monthly_df.head()

,Date,State,Total_cases,Total_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,1.0,0.0,1.0


In [41]:
# add death and recovery percentage columns. change NA values to zero
covid_monthly_df['Death_percent'] = round((covid_monthly_df.Total_death/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df['Recovery_percent'] = round((covid_monthly_df.Total_recovered/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df=covid_monthly_df.fillna(0)
covid_infec_df = covid_monthly_df.copy()
covid_monthly_df.tail()

,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
1615,2022-03,VT,105475.0,578.0,104897.0,0.55,99.45
1616,2022-03,WA,1437914.0,12183.0,1425731.0,0.85,99.15
1617,2022-03,WI,1577659.0,13713.0,1563946.0,0.87,99.13
1618,2022-03,WV,494875.0,6550.0,488325.0,1.32,98.68
1619,2022-03,WY,155744.0,1749.0,153995.0,1.12,98.88


In [42]:
# save df as a csv file
covid_monthly_df.to_csv('../Data/monthly.csv')

## USA Covid-19 Infection Map Data

In [43]:
# since data is aggregated using rolling average, we only goinog to keep last month data.
covid_infec_df = covid_infec_df.drop_duplicates(subset=['State'], keep='last', inplace=False, ignore_index=True)
covid_infec_df.tail()

,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
55,2022-03,VT,105475.0,578.0,104897.0,0.55,99.45
56,2022-03,WA,1437914.0,12183.0,1425731.0,0.85,99.15
57,2022-03,WI,1577659.0,13713.0,1563946.0,0.87,99.13
58,2022-03,WV,494875.0,6550.0,488325.0,1.32,98.68
59,2022-03,WY,155744.0,1749.0,153995.0,1.12,98.88


## USA Covid-19 Vaccine Status by States

In [44]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("unsk-b7fc", limit=50000)

# Convert to pandas DataFrame and display
covid_vac_df = pd.DataFrame.from_records(results)
covid_vac_df.head()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus
0,2022-03-13T00:00:00.000,11,MD,15119910,601500,5148200,9370210,0,250094,292858,...,90.7,4497062,79.1,11594264,203990,266020,1501367,2667576,323801,4318
1,2022-03-13T00:00:00.000,11,TN,12340430,506800,4713540,7120090,0,180702,211469,...,65.4,3674160,57.2,9652677,150340,192202,1306741,2122273,233340,11806
2,2022-03-13T00:00:00.000,11,BP2,336130,16100,149420,170610,0,0,0,...,0,132792,0,307221,0,0,52485,66829,13478,0
3,2022-03-13T00:00:00.000,11,MH,90480,12600,56400,21480,0,116386,157716,...,43.5,24905,36,57827,83567,130755,19485,2722,2696,2
4,2022-03-13T00:00:00.000,11,NC,21048400,919500,7926360,12202540,0,200689,234248,...,87.8,6253154,63.3,16226027,164259,213077,2164570,3608028,480409,147


In [45]:
covid_vac_df.dtypes

date                               object
mmwr_week                          object
location                           object
distributed                        object
distributed_janssen                object
                                    ...  
distributed_per_100k_5plus         object
series_complete_moderna_5plus      object
series_complete_pfizer_5plus       object
series_complete_janssen_5plus      object
series_complete_unk_manuf_5plus    object
Length: 82, dtype: object

In [46]:
# add date columns which only has year and month for use with groupby later on.. 
covid_vac_df['Date'] = pd.to_datetime(covid_vac_df['date']).dt.to_period('M').astype(str)
covid_vac_df.tail()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus,Date
29395,2020-12-13T00:00:00.000,51,AS,3900,0,0,0,0,7003,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
29396,2020-12-13T00:00:00.000,51,US,13650,0,0,0,0,4,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
29397,2020-12-13T00:00:00.000,51,GU,3900,0,0,0,0,2353,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
29398,2020-12-13T00:00:00.000,51,VI,975,0,0,0,0,931,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
29399,2020-12-13T00:00:00.000,51,MP,4875,0,0,0,0,8570,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12


In [47]:
# convert objects to float
covid_vac_df["distributed"] =  covid_vac_df.distributed.astype(float)
covid_vac_df["dist_per_100k"] = covid_vac_df.dist_per_100k.astype(float)
covid_vac_df["administered"] =  covid_vac_df.administered.astype(float)
covid_vac_df["admin_per_100k"] = covid_vac_df.admin_per_100k.astype(float)

# select only the columns we gonna use and display df
covid_vac_df = covid_vac_df[["Date", "location", "distributed", "dist_per_100k", "administered", "admin_per_100k"]]
covid_vac_df.head()

,Date,location,distributed,dist_per_100k,administered,admin_per_100k
0,2022-03,MD,15119910.0,250094.0,11595204.0,191793.0
1,2022-03,TN,12340430.0,180702.0,9653556.0,141358.0
2,2022-03,BP2,336130.0,0.0,307223.0,0.0
3,2022-03,MH,90480.0,116386.0,57853.0,74418.0
4,2022-03,NC,21048400.0,200689.0,16229800.0,154745.0


In [48]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_vac_df = covid_vac_df.rename(columns={"location": "State", "distributed": "Distributed", "dist_per_100k": "Dist_per_100k", "administered": "Administered", "admin_per_100k": "Admin_per_100k"})
covid_vac_df.tail()

,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
29395,2020-12,AS,3900.0,7003.0,0.0,0.0
29396,2020-12,US,13650.0,4.0,0.0,0.0
29397,2020-12,GU,3900.0,2353.0,0.0,0.0
29398,2020-12,VI,975.0,931.0,0.0,0.0
29399,2020-12,MP,4875.0,8570.0,0.0,0.0


In [49]:
# since data is aggregated using rolling average, we only goinog to keep month Data.
covid_vac_df = covid_vac_df.drop_duplicates(subset=['State'], keep='first', inplace=False, ignore_index=True)
covid_vac_df.head()

,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
0,2022-03,MD,15119910.0,250094.0,11595204.0,191793.0
1,2022-03,TN,12340430.0,180702.0,9653556.0,141358.0
2,2022-03,BP2,336130.0,0.0,307223.0,0.0
3,2022-03,MH,90480.0,116386.0,57853.0,74418.0
4,2022-03,NC,21048400.0,200689.0,16229800.0,154745.0


In [50]:
covid_vac_df.dtypes

Date               object
State              object
Distributed       float64
Dist_per_100k     float64
Administered      float64
Admin_per_100k    float64
dtype: object

In [51]:
# save df as a csv file
covid_vac_df.to_csv('../Data/vaccine.csv')

## USA Total Cases and Death by State

In [21]:
# upload data file and read it ito a pandas dataframe
# file = "../Data/US_COVID-19_Deaths.csv"
# covid_stats_df = pd.read_csv(file)
# covid_stats_df.head()

In [22]:
# select relevant columns only
# covid_stats_df = covid_stats_df[['State/Territory', 'Total Cases', 'Case Rate per 100000', 'Total Deaths', 'Death Rate per 100000']]
# covid_stats_df.head()

In [23]:
# add a state abbreviation column
# covid_stats_df['State'] = state_df['state']
# covid_stats_df.head()

In [24]:
# rename columns and drop any column with NA
# covid_stats_df = covid_stats_df.rename(columns={"state": "State", "Total Cases": "Total_cases", "Case Rate per 100000": "Case_rate_per_100k", "Total Deaths": "Total_deaths", "Death Rate per 100000": "Death_rate_per_100k"})
# covid_stats_df = covid_stats_df[["State", "Total_cases", "Case_rate_per_100k", "Total_deaths", "Death_rate_per_100k"]].fillna(0)
# covid_stats_df.tail()

## USA COVID-19 data with latitude and longtitude to create interactive map

In [25]:
# upload file and read to pandas dataframe
# file = "../Data/US_COVID-19_stats.csv"
# us_covid_df = pd.read_csv(file)
# us_covid_df.head()

In [26]:
# select relevant columns
# us_covid_df = us_covid_df[["Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Mortality_Rate"]]
# us_covid_df.head()

In [27]:
# add state column to be filled with state abbreviation
# us_covid_df["State"] = "" 
# us_covid_df.head()

In [28]:
# # upload States and their abbreviation
# us_state_abbrev = {
    
#     'Alabama': 'AL',
#     'Alaska': 'AK',
#     'American Samoa': 'AS',
#     'Arizona': 'AZ',
#     'Arkansas': 'AR',
#     'California': 'CA',
#     'Colorado': 'CO',
#     'Connecticut': 'CT',
#     'Delaware': 'DE',
#     'Diamond Princess': 'DP',
#     'District of Columbia': 'DC',
#     'Florida': 'FL',
#     'Georgia': 'GA',
#     'Grand Princess': 'GP',
#     'Guam': 'GU',
#     'Hawaii': 'HI',
#     'Idaho': 'ID',
#     'Illinois': 'IL',
#     'Indiana': 'IN',
#     'Iowa': 'IA',
#     'Kansas': 'KS',
#     'Kentucky': 'KY',
#     'Louisiana': 'LA',
#     'Maine': 'ME',
#     'Maryland': 'MD',
#     'Massachusetts': 'MA',
#     'Michigan': 'MI',
#     'Minnesota': 'MN',
#     'Mississippi': 'MS',
#     'Missouri': 'MO',
#     'Montana': 'MT',
#     'Nebraska': 'NE',
#     'Nevada': 'NV',
#     'New Hampshire': 'NH',
#     'New Jersey': 'NJ',
#     'New Mexico': 'NM',
#     'New York': 'NY',
#     'North Carolina': 'NC',
#     'North Dakota': 'ND',
#     'Northern Mariana Islands':'MP',
#     'Ohio': 'OH',
#     'Oklahoma': 'OK',
#     'Oregon': 'OR',
#     'Pennsylvania': 'PA',
#     'Puerto Rico': 'PR',
#     'Rhode Island': 'RI',
#     'South Carolina': 'SC',
#     'South Dakota': 'SD',
#     'Tennessee': 'TN',
#     'Texas': 'TX',
#     'Utah': 'UT',
#     'Vermont': 'VT',
#     'Virgin Islands': 'VI',
#     'Virginia': 'VA',
#     'Washington': 'WA',
#     'West Virginia': 'WV',
#     'Wisconsin': 'WI',
#     'Wyoming': 'WY'
# }
# abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [29]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     us_covid_df.loc[index, "State"] = us_state_abbrev[row[0]]

# us_covid_df.head()

In [ ]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long_"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long_"] = 117.5676

    # if row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [30]:
# rename columns and add recovery percent column
# us_covid_df = us_covid_df.rename(columns={"Long_": "Long", "Mortality_Rate":"Death_percent"})
# us_covid_df["Recovery_percent"]=us_covid_df.Recovered/us_covid_df.Confirmed
# us_covid_df = us_covid_df[["State", "Lat", "Long", "Death_percent", "Recovery_percent"]].fillna(0)
# us_covid_df.head()

In [32]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long"] = 117.5676

    # elif row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [33]:
# merge two df into one dataframe using left joint.
# us_covid_combine_df = pd.merge(covid_stats_df, us_covid_df, how='left', on=['State','State'])
# us_covid_combine_df.head()

In [34]:
# save to a csv file
# us_covid_combine_df.to_csv('../Data/states.csv')

## Add Geo property to states in old state file

In [35]:
# Add Lat and Long Columns
# geomap_df = covid_monthly_df.groupby('State').mean().reset_index()
# covid_monthly_df["Long"] = ""
# geomap_df.head()

## World COVID-19 Data by Country

In [36]:
# upload file and read into a pandas dataframe
# file = "../Data/World_COVID-19_Stats.csv"
# world_covid_df = pd.read_csv(file)
# world_covid_df.head()

In [37]:
# select relevant columns only
# world_covid_df = world_covid_df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Case-Fatality_Ratio"]]
# world_covid_df.head()

In [38]:
# rename columns and add a recovery percent column
# world_covid_df = world_covid_df.rename(columns={"Long_": "Long", "Case-Fatality_Ratio":"Death_percent", "Country_Region": "Country"})
# world_covid_df["Recovery_percent"]=world_covid_df.Recovered/world_covid_df.Confirmed
# world_covid_df.head()

In [39]:
# add missing geoinfo to country
# for index, row in world_covid_df.iterrows(): 
#     if row["Country"] == "Canada":
#         world_covid_df.loc[index, "Lat"] = 56.1304
#         world_covid_df.loc[index, "Long"] = 106.3468
# world_covid_df

In [40]:
# save file as a csv
# world_covid_df.to_csv('../Data/world.csv')

## build SQlite DataBase

In [52]:
# import modules
import sqlite3
from sqlite3 import Error

# series of functions to  build database tables in sqlite database.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def drop_table(conn, drop_table_sql):
    """ drop a table from the drop_table_sql statement
    :param conn: Connection object
    :param drop_table_sql: a drop TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(drop_table_sql)
    except Error as e:
        print(e)

# build empty tables with correct columns names
def main():		
    database = r"../DataBase/covid-19.db"
    sql_create_monthly_table = """CREATE TABLE monthly(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Total_cases float,
                                        Total_death float,
                                        Total_recovered float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_monthly_table = """DROP TABLE if exists monthly;"""

    sql_create_infection_table = """CREATE TABLE infection(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Total_cases float,
                                        Total_death float,
                                        Total_recovered float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_infection_table = """DROP TABLE if exists infection;"""

    sql_create_vaccine_table = """CREATE TABLE vaccine(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Distributed float,
                                        Dist_per_100k float,
                                        Administered float,
                                        Admin_per_100k float
                                    ); """

    sql_drop_vaccine_table = """DROP TABLE if exists vaccine;"""
    
    # sql_create_states_table = """CREATE TABLE states(
    #                                     id INTEGER PRIMARY KEY AUTOINCREMENT,
    #                                     State text,
    #                                     Total_cases float,
    #                                     Case_rate_per_100k float,
    #                                     Total_deaths float,
    #                                     Death_rate_per_100k float,
    #                                     Lat float,
    #                                     Long float,
    #                                     Death_percent float,
    #                                     Recovery_percent float
    #                                 );"""

    # sql_drop_states_table = """DROP TABLE if exists states;"""    

    # sql_create_world_table = """CREATE TABLE world(
    #                                     id INTEGER PRIMARY KEY AUTOINCREMENT,
    #                                     Country text,
    #                                     Lat float,
    #                                     Long float,
    #                                     Confirmed float,
    #                                     Deaths float,
    #                                     Recovered float,
    #                                     Recovery_percent float,
    #                                     Death_percent float
    #                                 );"""

    # sql_drop_world_table = """DROP TABLE if exists world;"""

    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create covid_monthly table
        drop_table(conn, sql_drop_monthly_table)
        create_table(conn, sql_create_monthly_table)

        # create covid_infection table
        drop_table(conn, sql_drop_infection_table)
        create_table(conn, sql_create_infection_table)

        # create covid_vaccine table
        drop_table(conn, sql_drop_vaccine_table)
        create_table(conn, sql_create_vaccine_table)

        # create covid_states table
        # drop_table(conn, sql_drop_states_table)
        # create_table(conn, sql_create_states_table)

        #create covid_world table
        # drop_table(conn, sql_drop_world_table)
        # create_table(conn, sql_create_world_table)
        
    # error handling
    else:
        print("Error! cannot create the database connection.")
if __name__ == '__main__':
    main()

In [53]:
# create engine connection to database
engine = create_engine('sqlite:///../DataBase/covid-19.db')

# # populate tables with information from dataframes
covid_monthly_df.to_sql(name='monthly', con=engine, if_exists = "append", index=False)
covid_infec_df.to_sql(name='infection', con=engine, if_exists = "append", index=False)
covid_vac_df.to_sql(name='vaccine', con=engine, if_exists = "append", index=False)
# world_covid_df.to_sql(name='world', con=engine, if_exists = "append", index= False)
# us_covid_combine_df.to_sql(name='states', con=engine, if_exists = "append", index=False)

In [54]:
# querry monthly data from database. to test and make sure tables are working
pd.read_sql('select * from monthly', engine)

,id,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,1,2020-01,AK,0.0,0.0,0.0,0.00,0.00
1,2,2020-01,AL,0.0,0.0,0.0,0.00,0.00
2,3,2020-01,AR,0.0,0.0,0.0,0.00,0.00
3,4,2020-01,AS,0.0,0.0,0.0,0.00,0.00
4,5,2020-01,AZ,1.0,0.0,1.0,0.00,100.00
...,...,...,...,...,...,...,...,...
1615,1616,2022-03,VT,105475.0,578.0,104897.0,0.55,99.45
1616,1617,2022-03,WA,1437914.0,12183.0,1425731.0,0.85,99.15
1617,1618,2022-03,WI,1577659.0,13713.0,1563946.0,0.87,99.13
1618,1619,2022-03,WV,494875.0,6550.0,488325.0,1.32,98.68


In [55]:
pd.read_sql('select * from vaccine', engine)

,id,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
0,1,2022-03,MD,15119910.0,250094.0,11595204.0,191793.0
1,2,2022-03,TN,12340430.0,180702.0,9653556.0,141358.0
2,3,2022-03,BP2,336130.0,0.0,307223.0,0.0
3,4,2022-03,MH,90480.0,116386.0,57853.0,74418.0
4,5,2022-03,NC,21048400.0,200689.0,16229800.0,154745.0
...,...,...,...,...,...,...,...
61,62,2022-03,WI,11139045.0,191313.0,9927645.0,170507.0
62,63,2022-03,CA,86541425.0,219024.0,72105757.0,182490.0
63,64,2022-03,IL,26346645.0,207915.0,21908184.0,172889.0
64,65,2022-02,RP,43990.0,204301.0,44526.0,206790.0


In [56]:
pd.read_sql('select * from infection', engine)

,id,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,1,2022-03,AK,235505.0,1168.0,234337.0,0.50,99.50
1,2,2022-03,AL,1289351.0,18890.0,1270461.0,1.47,98.53
2,3,2022-03,AR,825653.0,10864.0,814789.0,1.32,98.68
3,4,2022-03,AS,443.0,0.0,443.0,0.00,100.00
4,5,2022-03,AZ,1987318.0,28090.0,1959228.0,1.41,98.59
5,6,2022-03,CA,9019412.0,86387.0,8933025.0,0.96,99.04
6,7,2022-03,CO,1325063.0,11840.0,1313224.0,0.89,99.11
7,8,2022-03,CT,729697.0,10648.0,719049.0,1.46,98.54
8,9,2022-03,DC,135223.0,1326.0,133897.0,0.98,99.02
9,10,2022-03,DE,258024.0,2753.0,255271.0,1.07,98.93


In [45]:
# make an API call to make sure Flask app is working 
import requests
url="http://127.0.0.1:5000/api/v1.0/monthly"
resp=requests.get(url)
resp

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/v1.0/monthly (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022F371938D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
data = resp.json()
data